In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from sklearn.neural_network import MLPRegressor
import pickle as pickle

from sklearn.model_selection import train_test_split

In [ ]:
with open("bsc-3m/traj_01_elab.pkl", "rb") as f:
    df = pickle.load(f)

In [ ]:
cols=["id","class","xs","ys", "xs_640x360", "ys_640x360", "times", "frame_in", "frame_out", "num_frames"]
data = df[cols]
data["class"].unique()

In [ ]:
#id, class, xs, ys (or actually xs_640x360, ys_640x360 to plot it on the video/frame)
cols=["id","class","xs","ys", "xs_640x360", "ys_640x360", "times", "frame_in", "frame_out", "num_frames"]
data = df[cols]
data = data.loc[(data['class'] == 'Car') | (data['class'] == 'Van') | (data['class'] == 'Light Truck') | (data['class'] == 'Motorcycle')] # select cars

### features
    x_coordinate x # normalize
    y_coordinate x
    x_direction  x more!
    y_direction  x
    x_dest       x
    y_dest       x
    light_color # how
    surrounding_traffic?
    time?

### target
    next_x       x as vector please
    next_y       x

In [ ]:
def format_data(data, step_size):
    # position
    x_pos = np.array([])
    y_pos = np.array([])

    # target position
    x_tar = np.array([])
    y_tar = np.array([])

    # direction vectors
    x_vec = np.array([])
    y_vec = np.array([])
    
    # direction vectors
    x_vec2 = np.array([])
    y_vec2 = np.array([])

    # destination
    x_dest = np.array([])
    y_dest = np.array([])

    # times
    times = np.array([])

    euc = np.array([])
    
    # frames
    frames = np.array([])

    for i in range(len(data)):
        # current row
        a = np.array(data['xs'].iloc[i][::step_size])
        b = np.array(data['ys'].iloc[i][::step_size])
        if len(a) <= 2:
            continue
        

        # compute position
        x_pos = np.concatenate((x_pos, a[:-1]))
        y_pos = np.concatenate((y_pos, b[:-1]))

        # compute target vector
        x_tar = np.concatenate((x_tar, a[1:]-a[:-1]))
        y_tar = np.concatenate((y_tar, b[1:]-b[:-1]))

        # compute direction vectors
        x_prev = np.concatenate(([a[0]], a[:-2]))
        y_prev = np.concatenate(([b[0]], b[:-2]))
        x_vec = np.concatenate((x_vec, (a[:-1] - x_prev)))
        y_vec = np.concatenate((y_vec, (b[:-1] - y_prev)))
        
        # compute direction vectors
        x_prev2 = np.concatenate(([x_prev[0]], x_prev[:-1]))
        y_prev2 = np.concatenate(([y_prev[0]], y_prev[:-1]))
        x_vec2 = np.concatenate((x_vec2, (x_prev - x_prev2)))
        y_vec2 = np.concatenate((y_vec2, (y_prev - y_prev2)))

        # destination
        x_dest = np.concatenate((x_dest, [a[-1]]*(len(a)-1)))
        y_dest = np.concatenate((y_dest, [b[-1]]*(len(b)-1)))
        
        euc_x = np.array([a[-1]]*len(a)) - a
        euc_y = np.array([b[-1]]*len(b)) - b
        euc_i = np.linalg.norm(np.vstack((euc_x, euc_y)), axis=0)[:-1]
        euc = np.concatenate((euc, euc_i))
        
    df = pd.DataFrame({
                        'x_pos' : x_pos, 'y_pos' : y_pos,
                        'x_vec' : x_vec, 'y_vec' : y_vec,
                        'x_vec2' : x_vec2, 'y_vec2' : y_vec2,
                        'x_tar' : x_tar, 'y_tar' : y_tar,
                        'x_dest' : x_dest, 'y_dest' : y_dest,
                        'euc' : euc
                      })
    
    df = df.loc[df['euc']!=0]
    
    return df

In [ ]:
df_train, df_val = train_test_split(data, test_size=0.2, random_state=1)

df_out = format_data(df_train, 15)
with open('train.pkl', 'wb') as f:
    pickle.dump(df_out, f)


df_out = format_data(df_val, 15)
with open('val.pkl', 'wb') as f:
    pickle.dump(df_out, f)